In [1]:
import torch
from utility import image_util
import os
from voc12 import my_dataloader
from torch.utils.data import DataLoader
from misc import imutils
import numpy as np
import importlib

In [2]:
img_name_list_path = "voc12/val.txt"
voc12_root = "../Dataset/VOC2012/"
cam_root = "../result/val_cam/"
depth_root = "../result/depth_img/"

cam_network = "net.resnet50_cam"


In [3]:
dataset = my_dataloader.VOC12_Depth_CropClassificationDatasetMSF(img_name_list_path, voc12_root, cam_root, depth_root, scales = (1.0, 1.5, 0.5, 2.0))

In [4]:
data_loader = DataLoader(dataset)

In [5]:
pack = next(iter(data_loader))

crop_boxes = pack['crop_boxes']
org_size = pack['size']

strided_org_size = imutils.get_strided_size(org_size, 4)

crop_boxes = [tuple(int(t.item()) for t in crop_box) for crop_box in crop_boxes]
strided_crop_boxes = image_util.resize_bbox_list(crop_boxes, org_size, strided_org_size)
print(crop_boxes)
print(strided_crop_boxes)


[(98, 44, 363, 207), (0, 132, 315, 276)]
[(24, 11, 91, 51), (0, 33, 79, 69)]


/Users/yunsu/Desktop/Desktop/SKKU/CVML/Code/voc_cam/misc/imutils.py:177: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return ((orig_size[0]-1)//stride+1, (orig_size[1]-1)//stride+1)


In [7]:
img_name = pack['name'][0]
msf = pack['msf_img_list']
org_size = pack['size']
strided_org_size = imutils.get_strided_size(org_size, 4)

org_cam = pack['cam'][0].cpu().numpy()
org_high_res = pack['high_res'][0].cpu().numpy()
crop_labels = pack['crop_labels']
crop_boxes = pack['crop_boxes']
crop_sizes = pack['crop_sizes']
scale_crop_sizes = pack['scale_crop_sizes']

crop_boxes = [tuple(int(t.item()) for t in crop_box) for crop_box in crop_boxes]

strided_crop_boxes = image_util.resize_bbox_list(crop_boxes, org_size, strided_org_size)

cam_list = []
highres_cam_list = []

key = torch.sum(torch.cat(crop_labels, dim=0), dim=0)
key = torch.nonzero(key)[:, 0]

In [10]:
for idx, msf_img in enumerate(msf):
                
    msf_img = msf[idx]
    label = crop_labels[idx][0]

    scale_crop_size = scale_crop_sizes[idx]
    org_crop_size = crop_sizes[idx]

    scale_strided_size = imutils.get_strided_size(scale_crop_size, 4)
    scale_strided_up_size = imutils.get_strided_up_size(size, 16)

    
    
    valid_cat = torch.nonzero(label)[:, 0]
    
    
    
print(crop_sizes)
print(scale_crop_sizes)


[[tensor([163]), tensor([265])], [tensor([144]), tensor([315])]]
[[tensor([293]), tensor([477])], [tensor([173]), tensor([378])]]
(36, 79)
(144, 320)
